# Model validation: Predict Growth rate with limited NO3 uptake

Data from Rafay et al., 2020 (DOI: 10.3389/fmars.2020.00077)

### Directories

In [1]:
GEM_folder = "..\\..\\..\\..\\" # to update if necessary

In [2]:
modelfile = "model\\iMgadit23.json"

### Imports

In [3]:
import cobra
from cobra.flux_analysis.parsimonious import pfba
import json
import pandas as pd
import numpy as np
from os.path import join as __join
from matplotlib import pyplot as plt


In [4]:
cobra.show_versions()


System Information
OS         Windows
OS-release      10
Python      3.6.13

Package Versions
appdirs                                      1.4.4
black ; extra == 'development'       not installed
bumpversion ; extra == 'development' not installed
cobra                                       0.21.0
depinfo                                      1.7.0
diskcache                                    5.6.1
future                                      0.18.3
httpx                                       0.22.0
importlib-resources                          5.4.0
isort ; extra == 'development'       not installed
numpy                                       1.19.5
optlang                                      1.4.2
pandas                                       1.1.5
pip                                         21.2.2
pydantic                                     1.9.2
python-libsbml                              5.19.0
rich                                         6.2.0
ruamel.yaml                           

### Model

In [5]:
iMgadit = cobra.io.load_json_model(__join(GEM_folder, modelfile))

### Bounds

In [6]:
iMgadit.reactions.EX_photon_abs_.lower_bound =-1000
iMgadit.reactions.NGAM_c.lower_bound= 2.2
iMgadit.reactions.EX_nh4_.lower_bound = 0

### Simulations

In [7]:
no3_val = [1.33E-01,1.19E-01,1.24E-01]

In [8]:
obj_rxn = "BIOMASS_biomass_WT526_c"

In [9]:
df_res = pd.DataFrame()
for no3 in no3_val:
    iMgadit.reactions.EX_no3_.lower_bound = - no3
    iMgadit.objective = obj_rxn
    solution = pfba(iMgadit)
    df_res.loc[no3,"Growth"] = solution["BIOMASS_biomass_WT526_c"]
    df_res.loc[no3,"NO3"] = solution["EX_no3_"]
    df_res.loc[no3,"NO2"] = solution["EX_no2_"]
    df_res.loc[no3,"NH4"] = solution["EX_nh4_"]
    df_res.loc[no3,"CO2"] = solution["EX_co2_"]
    df_res.loc[no3,"O2"] = solution["EX_o2_"]
    df_res.loc[no3,"Photon"] = solution["EX_photon_abs_"]

In [10]:
df_res

,Growth,NO3,NO2,NH4,CO2,O2,Photon
0.133,0.026851,-0.133,0.0,0.0,-1.143048,1.561476,-22.979043
0.119,0.024025,-0.119,0.0,0.0,-1.022727,1.397110,-21.073770
0.124,0.025034,-0.124,0.0,0.0,-1.065699,1.455812,-21.755318


In [11]:
df_res.to_csv("Res_growth_rates_Rafay2020_data.csv",sep=";")